***Jay Frothingham***

# Final Project

____

Dataset source: [JPL Solar System Dynamics Databse](https://ssd.jpl.nasa.gov/?phys_data)

## Setup

In [ ]:
# import statements
import pandas as pd
import numpy as np
import scipy.optimize as opt
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats
import seaborn as sb # a new plotting library

In [ ]:
# these set the pandas defaults so that it will print ALL values, even for very long lists and large dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# filtering function
def filter(dataframe, col_name, value):
    """
    This function takes a dataframe, column name, and value.
    Returns a new dataframe that only includes rows where col_name==value.
    """
    filtered = dataframe[(dataframe[col_name]==value)]
    
    return filtered

In [ ]:
## read in the dataset here
data = pd.read_csv('comet_data.csv')
print(data.shape)

# print first few rows as a test
data[0:3]

## Basic info and data dictionary

In [ ]:
# what columns does this dataset include?
data.columns

| Column Name | Quantity | Variable Type | Units or Number | How Derived/Notes |
| :---        |    :---   |   :--- | : --- | :--- |
| e           | Eccentricity | Numerical | unitless | how non-circular the orbit is |
| q           | Perihelion distance | Numerical | astronomical units (au) | nearest approach to the sun |
| i           | Inclination | Numerical | degrees (deg) | how tilted the plane of the orbit is |
| w           | Argument of perihelion | Numerical | degrees (deg) | how tilted the orbit is with respect to its foci closest to the sun |
| ad          | Aphelion distance | Numerical | astronomical units (au) | furthest approach to the sun |
| tp.cal      | Time of perihelion passage | Numerical | Barycentric Dynamical Time (TDB) | defines when the comet reaches its perihelion |
| per.y       | Sidereal orbit period | Numerical | years | how long one orbit takes |
| class       | Orbit classification | Categorical | 6 types (Comet COM, Chiron-type CTc, Encke-type ETc, Halley-type HTC, Jupiter-family JFc defined in relation to Jupiter's period, Jupiter-family JFC based on direct period) |
| n_obs_used  | Number of observations of all types used in fit | Numerical | observations | 
| M1          | Comet total magnitude parameter | Numerical | magnitudes | brightness of the comet as a whole
| K1          | Comet total magnitude slope parameter | Numerical | magnitudes | characterizes whole comet brightness over time |
| M2          | Comet nuclear magnitude parameter | Numerical | magnitudes | brightness of the comet's core |
| K2          | Comet nuclear magnitude slope parameter | Numerical | magnitudes | characterizes comet core brightness over time |
| rot_per     | Rotation period | Numerical | hours (h) | time it takes for the comet to rotate about its axis |

In [ ]:
# custom function to print information about any particular numerical column
def num_info(array):
    """
    Given an array of numerical data, print the number of values, number of nonzero values, number of NaNs, min value, and max value.
    """
    print('\t Total values in dataset:', np.size(array))
    print('\t Nonzero values in dataset:', np.count_nonzero(array))
    print('\t NaN values in dataset:', np.count_nonzero(np.isnan(array)))
    print('\t Min value:', np.nanmin(array))
    print('\t Max value:', np.nanmax(array))

In [ ]:
# exploration of orbital period
orb_per = data['per.y'].values
print('SIDEREAL ORBITAL PERIOD')
num_info(orb_per)

In [ ]:
# exploration of perihelion distance
per_dist = data['q'].values

print('PERIHELION DISTANCE')
num_info(per_dist)

In [ ]:
# exploration of aphelion distance 
aph_dist = data['ad'].values

print('APHELION DISTANCE')
num_info(aph_dist)

In [ ]:
# exploration of orbital classification - categorical data
orb_class = data['class'].values
print('Total values in dataset:', np.size(orb_class))
print('Unique values in dataset:', np.unique(orb_class))

In [ ]:
# exploration of total magnitude
m1 = data['M1'].values
print('TOTAL MAGNITUDE')
num_info(m1)

In [ ]:
# exploration of nuclear magnitude
m2 = data['M2'].values
print('NUCLEAR MAGNITUDE')
num_info(m2)

In [ ]:
# exploration of how many observations went into orbit fits
obs = data['n_obs_used'].values
print('NUMBER OF OBSERVATIONS USED FOR FITTING')
num_info(obs)

## Histograms and distributions

In [ ]:
# perihelion and aphelion exploration

plt.figure(figsize=(10,5))

# perihelion distance histogram and distribution
plt.subplot(1,2,1)
plt.title('Perihelion distance')
plt.ylabel('frequency in dataset')
plt.xlabel('astronomical units (au)')
b = np.arange(np.nanmin(per_dist), np.nanmax(per_dist)+1)
plt.hist(per_dist, bins=100, label='100 bins');
plt.legend(frameon=False)

print('perihelion mean:', np.nanmean(per_dist), 'au')
print('perihelion std:', np.nanstd(per_dist), 'au')

# aphelion distance histogram and distribution
plt.subplot(1,2,2)
plt.title('Aphelion distance')
plt.ylabel('frequency in dataset')
plt.xlabel('astronomical units (au)')
b = np.arange(np.nanmin(aph_dist), np.nanmax(aph_dist)+1)
plt.yscale('log')
plt.hist(aph_dist, bins=b, label=str(b.size)+'bins');
plt.legend(frameon=False)

print('aphelion mean:', np.nanmean(aph_dist), 'au')
print('aphelion std:', np.nanstd(aph_dist), 'au')

In [ ]:
# orbital period and classification exploration

plt.figure(figsize=(10,5))

# orbital classification histogram
plt.subplot(1,2,1)
plt.title('Orbit classifications')
plt.ylabel('frequency in dataset')
plt.xlabel('abbreviated orbit classes')
b = np.arange(0, np.size(np.unique(orb_class))+1)
#plt.yscale('log')
plt.hist(orb_class, bins=b);

# orbital period histogram
plt.subplot(1,2,2)
plt.title('Sidereal orbital period')
plt.ylabel('frequency in dataset')
plt.xlabel('years')
plt.xscale('log')
b = np.arange(np.nanmin(orb_per), np.nanmax(orb_per)+1)
plt.hist(orb_per, bins=b, label=str(b.size)+' bins');
plt.legend(frameon=False)

In [ ]:
print('Frequency of orbital classes in dataset')
print('COM:',np.size(np.where(orb_class == 'COM')))
print('CTc:',np.size(np.where(orb_class == 'CTc')))
print('ETc:',np.size(np.where(orb_class == 'ETc')))
print('HTC:',np.size(np.where(orb_class == 'HTC')))
print('JFc:',np.size(np.where(orb_class == 'JFc')))
print('JFC:',np.size(np.where(orb_class == 'JFC')))

In [ ]:
# total and nuclear magnitude exploration

plt.figure(figsize=(10,5))

# total magnitude histogram
plt.subplot(1,2,1)
plt.title('Total comet magnitude')
plt.ylabel('frequency in dataset')
plt.xlabel('magnitudes')
b = np.arange(np.nanmin(m1), np.nanmax(m1)+1)
plt.gca().invert_xaxis() # higher mag = dimmer; smaller mag = brighter
plt.hist(m1, bins=b, label=str(b.size)+' bins');
plt.legend(frameon=False)

print('total magnitude mean:', np.nanmean(m1))
print('total magnitude std:', np.nanstd(m1))

# nuclear magnitude histogram
plt.subplot(1,2,2)
plt.title('Comet nuclear magnitude')
plt.ylabel('frequency in dataset')
plt.xlabel('magnitudes')
b = np.arange(np.nanmin(m2), np.nanmax(m2)+1)
plt.gca().invert_xaxis()
plt.hist(m2, bins=b, label=str(b.size)+' bins');
plt.legend(frameon=False)

print('nuclear magnitude mean:', np.nanmean(m2))
print('nuclear magnitude std:', np.nanstd(m2))

## Investigations

Is comet brightness related to either orbital period or aphelion and perihelion distances, and are there different relationships for total comet magnitude versus nuclear magnitude?

In [ ]:
# separate into populations based on orbital class
COM = filter(data, 'class', 'COM')
CTc = filter(data, 'class', 'CTc')
ETc = filter(data, 'class', 'ETc')
HTC = filter(data, 'class', 'HTC')
JFc = filter(data, 'class', 'JFc')
JFC = filter(data, 'class', 'JFC')

In [ ]:
# scatterplot of aphelion vs perihelion, separated by orbit class

plt.ylabel('perihelion distance')
plt.xlabel('aphelion distance')
plt.xscale('log')
plt.scatter(COM['ad'].values, COM['q'].values, alpha=0.5, marker='*', label='COM')
plt.scatter(CTc['ad'].values, CTc['q'].values, alpha=0.5, marker='*', label='CTc')
plt.scatter(ETc['ad'].values, ETc['q'].values, alpha=0.5, marker='*', label='ETc')
plt.scatter(HTC['ad'].values, HTC['q'].values, alpha=0.5, marker='*', label='HTC')
plt.scatter(JFc['ad'].values, JFc['q'].values, alpha=0.5, marker='*', label='JFc')
plt.scatter(JFC['ad'].values, JFC['q'].values, alpha=0.5, marker='*', label='JFC')
plt.legend(frameon=False)

In [ ]:
# arrays of number of observations for different classes


In [ ]:
# exploratory plots of different combos of magnitudes and distances
# separated by orbital class

plt.figure(figsize=(10,10))

plt.subplot(2,2,1)
plt.title('total magnitude and aphelion distance')
plt.xlabel('total magnitude')
plt.ylabel('aphelion distance in au')
plt.yscale('log')
plt.scatter(COM['M1'].values, COM['ad'].values, s=0.01*COM['n_obs_used'].values, alpha=0.5, marker='*', label='COM')
plt.scatter(CTc['M1'].values, CTc['ad'].values, s=0.01*CTc['n_obs_used'].values,alpha=0.5, marker='*', label='CTc')
plt.scatter(ETc['M1'].values, ETc['ad'].values, s=0.01*ETc['n_obs_used'].values,alpha=0.5, marker='*', label='ETc')
plt.scatter(HTC['M1'].values, HTC['ad'].values, s=0.01*HTC['n_obs_used'].values,alpha=0.5, marker='*', label='HTC')
plt.scatter(JFc['M1'].values, JFc['ad'].values, s=0.01*JFc['n_obs_used'].values,alpha=0.5, marker='*', label='JFc')
plt.scatter(JFC['M1'].values, JFC['ad'].values, s=0.01*JFC['n_obs_used'].values,alpha=0.5, marker='*', label='JFC')
plt.legend(frameon=False)

plt.subplot(2,2,2)
plt.title('nuclear magnitude and aphelion distance')
plt.xlabel('nuclear magnitude')
plt.ylabel('aphelion distance in au')
plt.yscale('log')
plt.scatter(COM['M2'].values, COM['ad'].values, s=0.01*COM['n_obs_used'].values, alpha=0.5, marker='*', label='COM')
plt.scatter(CTc['M2'].values, CTc['ad'].values, s=0.01*CTc['n_obs_used'].values,alpha=0.5, marker='*', label='CTc')
plt.scatter(ETc['M2'].values, ETc['ad'].values, s=0.01*ETc['n_obs_used'].values,alpha=0.5, marker='*', label='ETc')
plt.scatter(HTC['M2'].values, HTC['ad'].values, s=0.01*HTC['n_obs_used'].values,alpha=0.5, marker='*', label='HTC')
plt.scatter(JFc['M2'].values, JFc['ad'].values, s=0.01*JFc['n_obs_used'].values,alpha=0.5, marker='*', label='JFc')
plt.scatter(JFC['M2'].values, JFC['ad'].values, s=0.01*JFC['n_obs_used'].values,alpha=0.5, marker='*', label='JFC')
plt.legend(frameon=False)

plt.subplot(2,2,3)
plt.title('total magnitude and perihelion distance')
plt.xlabel('total magnitude')
plt.ylabel('perihelion distance in au')
plt.yscale('log')
plt.scatter(COM['M1'].values, COM['q'].values, s=0.01*COM['n_obs_used'].values, alpha=0.5, marker='*', label='COM')
plt.scatter(CTc['M1'].values, CTc['q'].values, s=0.01*CTc['n_obs_used'].values,alpha=0.5, marker='*', label='CTc')
plt.scatter(ETc['M1'].values, ETc['q'].values, s=0.01*ETc['n_obs_used'].values,alpha=0.5, marker='*', label='ETc')
plt.scatter(HTC['M1'].values, HTC['q'].values, s=0.01*HTC['n_obs_used'].values,alpha=0.5, marker='*', label='HTC')
plt.scatter(JFc['M1'].values, JFc['q'].values, s=0.01*JFc['n_obs_used'].values,alpha=0.5, marker='*', label='JFc')
plt.scatter(JFC['M1'].values, JFC['q'].values, s=0.01*JFC['n_obs_used'].values,alpha=0.5, marker='*', label='JFC')
plt.legend(frameon=False)

plt.subplot(2,2,4)
plt.title('nuclear magnitude and perihelion distance')
plt.xlabel('nuclear magnitude')
plt.ylabel('perihelion distance in au')
plt.yscale('log')
plt.scatter(COM['M2'].values, COM['q'].values, s=0.01*COM['n_obs_used'].values, alpha=0.5, marker='*', label='COM')
plt.scatter(CTc['M2'].values, CTc['q'].values, s=0.01*CTc['n_obs_used'].values,alpha=0.5, marker='*', label='CTc')
plt.scatter(ETc['M2'].values, ETc['q'].values, s=0.01*ETc['n_obs_used'].values,alpha=0.5, marker='*', label='ETc')
plt.scatter(HTC['M2'].values, HTC['q'].values, s=0.01*HTC['n_obs_used'].values,alpha=0.5, marker='*', label='HTC')
plt.scatter(JFc['M2'].values, JFc['q'].values, s=0.01*JFc['n_obs_used'].values,alpha=0.5, marker='*', label='JFc')
plt.scatter(JFC['M2'].values, JFC['q'].values, s=0.01*JFC['n_obs_used'].values,alpha=0.5, marker='*', label='JFC')
plt.legend(frameon=False)

### The aphelion vs magnitude plots show a clear cutoff between orbit classes. This indicates a relationship between aphelion and orbital period.

In [ ]:
# scatterplot of orbital period vs perihelion and aphelion distances

plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.xlabel('sidereal orbital period [years]')
plt.ylabel('perihelion distance [au]')
plt.semilogx(orb_per, per_dist, '*')

plt.subplot(1,2,2)
plt.xlabel('sidereal orbital period [years]')
plt.ylabel('aphelion distance [au]')
plt.semilogx(orb_per, aph_dist, '*')

In [ ]:
# Kepler's third law: 
# square of orbital period is proportional to cube of semi-major axis

# aphelion distance is approximately equal to semi-major axis

# plot to check whether this is the relationship shown
kep_x = np.arange(np.nanmin(orb_per), np.nanmax(orb_per), 0.01)
kep_aph_dist = kep_x ** (2/3)

plt.title('Kepler\'s 3rd law with comets')
plt.xlabel('sidereal orbital period P [years]')
plt.ylabel('aphelion distance d [au]')
plt.semilogx(orb_per, aph_dist, 'b*', label='data')
plt.semilogx(kep_x, kep_aph_dist, label='d = P$^{2/3}$')
plt.semilogx(kep_x, 1.5*kep_aph_dist, label='d = 1.5 P$^{2/3}$')
plt.semilogx(kep_x, 2*kep_aph_dist, label='d = 2 P$^{2/3}$')
plt.legend(frameon=False)

### Looks like the comets in this dataset obey Kepler's third law with proportionality constant between 1 and 2.

In [ ]:
# exploratory plots of different combos of magnitudes and distances
# separated by orbital class

# plot repeated here as a reminder

plt.figure(figsize=(10,10))

plt.subplot(2,2,1)
plt.title('total magnitude and aphelion distance')
plt.xlabel('total magnitude')
plt.ylabel('aphelion distance in au')
plt.yscale('log')
plt.scatter(COM['M1'].values, COM['ad'].values, alpha=0.5, marker='*', label='COM')
plt.scatter(CTc['M1'].values, CTc['ad'].values, alpha=0.5, marker='*', label='CTc')
plt.scatter(ETc['M1'].values, ETc['ad'].values, alpha=0.5, marker='*', label='ETc')
plt.scatter(HTC['M1'].values, HTC['ad'].values, alpha=0.5, marker='*', label='HTC')
plt.scatter(JFc['M1'].values, JFc['ad'].values, alpha=0.5, marker='*', label='JFc')
plt.scatter(JFC['M1'].values, JFC['ad'].values, alpha=0.5, marker='*', label='JFC')
plt.legend(frameon=False)

plt.subplot(2,2,2)
plt.title('nuclear magnitude and aphelion distance')
plt.xlabel('nuclear magnitude')
plt.ylabel('aphelion distance in au')
plt.yscale('log')
plt.scatter(COM['M2'].values, COM['ad'].values, alpha=0.5, marker='*', label='COM')
plt.scatter(CTc['M2'].values, CTc['ad'].values, alpha=0.5, marker='*', label='CTc')
plt.scatter(ETc['M2'].values, ETc['ad'].values, alpha=0.5, marker='*', label='ETc')
plt.scatter(HTC['M2'].values, HTC['ad'].values, alpha=0.5, marker='*', label='HTC')
plt.scatter(JFc['M2'].values, JFc['ad'].values, alpha=0.5, marker='*', label='JFc')
plt.scatter(JFC['M2'].values, JFC['ad'].values, alpha=0.5, marker='*', label='JFC')
plt.legend(frameon=False)

plt.subplot(2,2,3)
plt.title('total magnitude and perihelion distance')
plt.xlabel('total magnitude')
plt.ylabel('perihelion distance in au')
plt.yscale('log')
plt.scatter(COM['M1'].values, COM['q'].values, alpha=0.5, marker='*', label='COM')
plt.scatter(CTc['M1'].values, CTc['q'].values, alpha=0.5, marker='*', label='CTc')
plt.scatter(ETc['M1'].values, ETc['q'].values, alpha=0.5, marker='*', label='ETc')
plt.scatter(HTC['M1'].values, HTC['q'].values, alpha=0.5, marker='*', label='HTC')
plt.scatter(JFc['M1'].values, JFc['q'].values, alpha=0.5, marker='*', label='JFc')
plt.scatter(JFC['M1'].values, JFC['q'].values, alpha=0.5, marker='*', label='JFC')
plt.legend(frameon=False)

plt.subplot(2,2,4)
plt.title('nuclear magnitude and perihelion distance')
plt.xlabel('nuclear magnitude')
plt.ylabel('perihelion distance in au')
plt.yscale('log')
plt.scatter(COM['M2'].values, COM['q'].values, alpha=0.5, marker='*', label='COM')
plt.scatter(CTc['M2'].values, CTc['q'].values, alpha=0.5, marker='*', label='CTc')
plt.scatter(ETc['M2'].values, ETc['q'].values, alpha=0.5, marker='*', label='ETc')
plt.scatter(HTC['M2'].values, HTC['q'].values, alpha=0.5, marker='*', label='HTC')
plt.scatter(JFc['M2'].values, JFc['q'].values, alpha=0.5, marker='*', label='JFc')
plt.scatter(JFC['M2'].values, JFC['q'].values, alpha=0.5, marker='*', label='JFC')
plt.legend(frameon=False)

### The perihelion vs magnitude plots show a potentially curve-fittable relationship, with the same trend but different slopes for nuclear magnitude and total magnitude.

In [ ]:
# scatterplot of nuclear magnitude vs total magnitude
plt.title('total magnitude and nuclear magnitude')
plt.xlabel('total magnitude')
plt.ylabel('nuclear magnitude')
plt.scatter(COM['M1'].values, COM['M2'].values, alpha=0.5, marker='*', label='COM')
plt.scatter(CTc['M1'].values, CTc['M2'].values, alpha=0.5, marker='*', label='CTc')
plt.scatter(ETc['M1'].values, ETc['M2'].values, alpha=0.5, marker='*', label='ETc')
plt.scatter(HTC['M1'].values, HTC['M2'].values, alpha=0.5, marker='*', label='HTC')
plt.scatter(JFc['M1'].values, JFc['M2'].values, alpha=0.5, marker='*', label='JFc')
plt.scatter(JFC['M1'].values, JFC['M2'].values, alpha=0.5, marker='*', label='JFC')
plt.legend(frameon=False)

In [ ]:
# curve-fitting to describe relationship between nuclear and total magnitude
quad = lambda x,a,b,c: a*(x**2) + b*x + c

mask_mag = ~np.isnan(m1) & ~np.isnan(m2)
m1_nonan = m1[mask_mag]
m2_nonan = m2[mask_mag]


fit = curve_fit(quad, m1_nonan, m2_nonan)
qfit_x = np.arange(np.nanmin(m1_nonan), np.nanmax(m1_nonan), 0.01)
qfit = quad(qfit_x, fit[0][0], fit[0][1], fit[0][2])

qfit_resids = m2_nonan - quad(m1_nonan, fit[0][0], fit[0][1], fit[0][2])

plt.figure(figsize=(10,4))

plt.subplot(1,2,2)
plt.title('residuals')
plt.xlabel('total magnitude')
plt.ylabel('difference between fit and data')
plt.plot(m1_nonan, qfit_resids, '*')
plt.axhline(0, ls='--')

plt.subplot(1,2,1)
plt.title('curvefit')
plt.xlabel('total magnitude')
plt.ylabel('nuclear magnitude')
plt.plot(m1_nonan, m2_nonan, '*',alpha=0.75, label='database')
plt.plot(qfit_x, qfit, 'r-',label=f'y={fit[0][0]:6.4}x$^2$+{fit[0][1]:6.4}x+{fit[0][2]:6.4}')
plt.legend(frameon=False)

In [ ]:
# define a function for nuclear magnitude from total magnitude
def nuc_from_total(tot, fit_param):
    """
    Takes an array of total magnitudes and an array of quadratic fit parameters.
    Returns calculated array of nuclear magnitudes.
    """
    return quad(tot, fit_param[0][0], fit_param[0][1], fit_param[0][2])

In [ ]:
# curve-fitting to describe relation between perihelion and nuclear magnitude

lin = lambda x,a,b: a*x + b

mask_per = ~np.isnan(m2) & ~np.isnan(per_dist)
m2_nonan_per = m2[mask_per]
per_dist_nonan = per_dist[mask_per]

fit_lin = curve_fit(lin, m2_nonan_per, per_dist_nonan)
fit_quad = curve_fit(quad, m2_nonan_per, per_dist_nonan)
fit_log = curve_fit(lin, m2_nonan_per, np.log10(per_dist_nonan))

fit_x = np.arange(np.nanmin(m2_nonan_per), np.nanmax(m2_nonan_per), 0.01)

lfit = lin(fit_x, fit_lin[0][0], fit_lin[0][1])
qfit = quad(fit_x, fit_quad[0][0], fit_quad[0][1], fit_quad[0][2])
logfit = lin(fit_x, fit_log[0][0], fit_log[0][1])

lfit_resids = per_dist_nonan - lin(m2_nonan_per, fit_lin[0][0], fit_lin[0][1])
qfit_resids = per_dist_nonan - quad(m2_nonan_per, fit_quad[0][0], fit_quad[0][1], fit_quad[0][2])
logfit_resids = per_dist_nonan - 10**(lin(m2_nonan_per, fit_log[0][0], fit_log[0][1]))

In [ ]:
plt.figure(figsize=(20,6))

plt.subplot(1,3,3)
plt.title('residuals')
plt.ylabel('difference between data and fit')
plt.xlabel('nuclear magnitude m2')
plt.plot(m2_nonan_per, lfit_resids, 'c*', alpha=0.75, label='linear fit residuals')
plt.plot(m2_nonan_per, qfit_resids, 'm*', alpha=0.75, label='quad fit residuals')
plt.plot(m2_nonan_per, logfit_resids, 'b*', label='log fit residuals')
plt.axhline(0, ls='--')
plt.legend(frameon=False)

plt.subplot(1,3,1)
plt.title('fitting on a linear scale')
plt.ylabel('perihelion distance in au')
plt.xlabel('nuclear magnitude m2')
plt.plot(m2_nonan_per, per_dist_nonan, '*',alpha=0.75, label='database')
plt.plot(fit_x, lfit, 'c-',label=f'y={fit_lin[0][0]:6.4}x+{fit_lin[0][1]:6.4}')
plt.plot(fit_x, qfit, 'm-',label=f'y={fit_quad[0][0]:6.4}x$^2$+{fit_quad[0][1]:6.4}x+{fit_quad[0][2]:6.4}')
plt.plot(fit_x, 10**logfit, 'b-', label=f'log$_1$$_0$(y)={fit_log[0][0]:6.4}x+{fit_log[0][1]:6.4}')

plt.legend(frameon=False)

plt.subplot(1,3,2)
plt.title('fitting on a log scale')
plt.ylabel('perihelion distance in au')
plt.xlabel('nuclear magnitude m2')
plt.yscale('log')
plt.plot(m2_nonan_per, per_dist_nonan, '*',alpha=0.75, label='database')
plt.plot(fit_x, lfit, 'c-',label=f'y={fit_lin[0][0]:6.4}x+{fit_lin[0][1]:6.4}')
plt.plot(fit_x, qfit, 'm-',label=f'y={fit_quad[0][0]:6.4}x$^2$+{fit_quad[0][1]:6.4}x+{fit_quad[0][2]:6.4}')
plt.plot(fit_x, 10**logfit, 'b-', label=f'log$_1$$_0$(y)={fit_log[0][0]:6.4}x+{fit_log[0][1]:6.4}')
plt.legend(frameon=False)

## None of the fits are significantly better than the others because there is so much spread in the data. Linear fit seems to have the best residuals - log fit skews high, quad fit skews low. Visually, though, I think log fit matches the shape of the data best.

### Monte Carlo simulation, assuming uniform probability of perihelion distances and a linear relationship between nuclear magnitude and perihelion distances

In [ ]:
def bestfits(n, xvals, yvals):
    """
    Draw an arbitrary number of random points from the distribution
    Return an array of best fit slopes and correlation coefficients
    """
    covmat = np.cov(xvals, yvals)
    num_pts = xvals.size
    slopes = np.zeros(n)
    r_vals = np.zeros(n)
    y_ints = np.zeros(n)

    for i in range(n):
        xval_calc, yval_calc = np.random.multivariate_normal((np.mean(xvals), np.mean(yvals)), covmat, num_pts).T
        r_vals[i], pval = stats.pearsonr(xval_calc, yval_calc)

        lin = lambda x,a,b: a*x + b
        (slopes[i], y_ints[i]), cov = opt.curve_fit(lin, xval_calc, yval_calc)

    return (slopes, r_vals)

In [ ]:
fit_slope = fit_lin[0][0]
test_mags = bestfits(1000, m2_nonan_per, per_dist_nonan)
test_slope = np.mean(test_mags[0])

b=np.arange(np.min(test_mags[0]), np.max(test_mags[0]),0.01).size

plt.figure(figsize=(8,4))

plt.title('Monte Carlo simulation: \n slopes of linear relation between nuclear magnitude and perihelion distance')
plt.ylabel('frequency')
plt.xlabel('slope')
plt.xlim([-0.5,0])
plt.hist(test_mags[0], label='random slope distribution; '+str(b)+' bins', bins=b)
plt.axvline(test_slope, c='b',label='distribution mean = '+str(np.round(test_slope,4)))
plt.axvline(fit_slope, c='r', label='slope fit to original data = '+str(np.round(fit_slope,4)))
plt.legend(frameon=False)

print('random slope distribution mean:', np.round(test_slope,4))
print('original data fit slope:', np.round(fit_slope,4))

### This produces a random distribution of nuclear magnitudes and perihelion distances with linear fits with normally-distributed slopes. The mean of the distribution is very similar to the slope of the line fit to the original data.

In [ ]:
# define a function for perihelion distance from nuclear magnitude
def per_from_nuc(nuc, fit_param):
    """
    Takes an array of nuclear magnitudes and an array of fit parameters.
    Uses log fit defined earlier in notebook.
    Returns calculated array of perihelion distances.
    """
    return 10**(lin(nuc, fit_param[0][0], fit_param[0][1]))

In [ ]:
# calculate what perihelion distances should be based on total magntidues -- this is model 1
calc_nuc_mag = nuc_from_total(m1_nonan, fit)
calc_peri = per_from_nuc(calc_nuc_mag, fit_log)

In [ ]:
# plot calculated perihelion distances vs total magnitudes and actual perihelion distances vs total magnitudes
plt.figure(figsize=(8,6))
plt.title('Calculated model compared to actual data')
plt.ylabel('perihelion distance [au]')
plt.xlabel('total magnitude')
plt.scatter(m1, per_dist, label='dataset')
plt.scatter(m1_nonan, calc_peri, label='calculated from log model')
plt.legend(frameon=False)

## Visually, this looks like a pretty good match! That suggests that the different relationships between perihelion distance and the two types of magnitude ARE an artifact of the relationship between total and nuclear magnitude.

In [ ]:
# curve-fit relationship between actual perihelion distance and total magnitude -- this is model 2


In [ ]:
# bayes/hypothesis test between the two models - is there agreement between the two?